In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
# Obtain Dataset
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -qq 'tiny-imagenet-200.zip'

--2021-09-11 04:31:13--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  20.2MB/s    in 13s     

2021-09-11 04:31:27 (18.1 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



# Loading Data Set and Assembling Training, Validation, and Test Sets

In [18]:
# Load training dataset, then split into training, validation, and test sets
import torch
import torchvision
import torchvision.transforms as transforms

BATCH_SIZE = 32

# Number of batches in Training, Validation, Test Sets
TRAINING_BATCHES = 1000
VALIDATION_BATCHES = 100
TEST_BATCHES = 200

# Load training data
train_dir = "./tiny-imagenet-200/train"
train_data = torchvision.datasets.ImageFolder(train_dir, transform=transforms.Compose([transforms.ToTensor()]))
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle = True)


# Loop through data loader and add batches and labels based on constants
training_set = []
validation_set = []
test_set = []
i = 0
for set_data in train_data_loader:
  i += 1
  if i <= TRAINING_BATCHES:
    training_set.append(set_data)
  
  elif i <= TRAINING_BATCHES + VALIDATION_BATCHES:
    validation_set.append(set_data)
  
  elif i <= TRAINING_BATCHES + VALIDATION_BATCHES + TEST_BATCHES:
    test_set.append(set_data)
  
  else:
    break

print(len(training_set))
print(len(validation_set))
print(len(test_set))


500
100
200


# Training Code



In [65]:
LR = 1e-4
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-6
EPOCHS = 10

model = torchvision.models.resnet50(pretrained=True, progress=True)
model.fc = torch.nn.Linear(in_features=2048, out_features=200)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

optimizers = {
    "Adam" : torch.optim.Adam(model.parameters(), lr = LR, weight_decay=WEIGHT_DECAY)
}

def train_model(optimizer_spec):
  loss_f = torch.nn.CrossEntropyLoss()
  optimizer = optimizers[optimizer_spec]
  epoch_loss = []
  for epoch in range (EPOCHS):
    print("EPOCH : ", epoch)
    model.train()
    for im, truth_labels in training_set:
      im = im.to(device)
      truth_labels = truth_labels.to(device)

      predicted_labels = model(im)
      loss = loss_f(predicted_labels, truth_labels)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      epoch_loss.append(loss.item())
      # accuracy = (predicted_labels.argmax(1) == truth_labels).float().sum().item()
      # print("TRAIN_ACCURACY : ", accuracy)
      train_set_accuracy = torch.FloatTensor(epoch_loss).mean().item()
    print("TRAIN_ACCURACY : ", train_set_accuracy)

    model.eval()
    accuracies = []
    for validation_im, validation_labels in validation_set:
      validation_im = validation_im.to(device)
      validation_labels = validation_labels.to(device)

      predicted_labels = model(validation_im).argmax(1)
      accuracy = (predicted_labels == validation_labels).float().mean().item()
      accuracies.append(accuracy)
    
    validation_set_accuracy = torch.FloatTensor(accuracies).mean().item()
    print("******VALIDATION ACCURACY*************** : ", validation_set_accuracy)


In [66]:
train_model("Adam")

EPOCH :  0
TRAIN_ACCURACY :  3.591287136077881
******VALIDATION ACCURACY*************** :  0.41187500953674316
EPOCH :  1
TRAIN_ACCURACY :  2.6111855506896973
******VALIDATION ACCURACY*************** :  0.46187499165534973
EPOCH :  2
TRAIN_ACCURACY :  1.9248934984207153
******VALIDATION ACCURACY*************** :  0.47312501072883606
EPOCH :  3
TRAIN_ACCURACY :  1.4792784452438354
******VALIDATION ACCURACY*************** :  0.47593748569488525
EPOCH :  4
TRAIN_ACCURACY :  1.1894357204437256
******VALIDATION ACCURACY*************** :  0.48625001311302185
EPOCH :  5
TRAIN_ACCURACY :  0.9924259185791016
******VALIDATION ACCURACY*************** :  0.49687498807907104
EPOCH :  6
TRAIN_ACCURACY :  0.8512016534805298
******VALIDATION ACCURACY*************** :  0.5018749833106995
EPOCH :  7
TRAIN_ACCURACY :  0.7450770139694214
******VALIDATION ACCURACY*************** :  0.49906250834465027
EPOCH :  8
TRAIN_ACCURACY :  0.6624607443809509
******VALIDATION ACCURACY*************** :  0.500625014305

In [ ]:
Resnet18 le 1e-4

EPOCH :  0
TRAINING_LOSS :  4.052446381568909
******VALIDATION ACCURACY*************** :  0.31468749046325684
EPOCH :  1
TRAINING_LOSS :  6.134171817779541
******VALIDATION ACCURACY*************** :  0.36937499046325684
EPOCH :  2
TRAINING_LOSS :  6.999579892873764
******VALIDATION ACCURACY*************** :  0.375
EPOCH :  3
TRAINING_LOSS :  7.234807624876499
******VALIDATION ACCURACY*************** :  0.37562501430511475
EPOCH :  4
TRAINING_LOSS :  7.292210902020336
******VALIDATION ACCURACY*************** :  0.3918749988079071
EPOCH :  5
TRAINING_LOSS :  7.315568085754291
******VALIDATION ACCURACY*************** :  0.39125001430511475
EPOCH :  6
TRAINING_LOSS :  7.32907930099219
******VALIDATION ACCURACY*************** :  0.39125001430511475
EPOCH :  7
TRAINING_LOSS :  7.33790581655316
******VALIDATION ACCURACY*************** :  0.3918749988079071
EPOCH :  8
TRAINING_LOSS :  7.343977942521218
******VALIDATION ACCURACY*************** :  0.3918749988079071
EPOCH :  9

Resnet 50 1e-4

Best Validation Accuracy : 0.48

EPOCH :  0
TRAIN_ACCURACY :  3.591287136077881
******VALIDATION ACCURACY*************** :  0.41187500953674316
EPOCH :  1
TRAIN_ACCURACY :  2.6111855506896973
******VALIDATION ACCURACY*************** :  0.46187499165534973
EPOCH :  2
TRAIN_ACCURACY :  1.9248934984207153
******VALIDATION ACCURACY*************** :  0.47312501072883606
EPOCH :  3
TRAIN_ACCURACY :  1.4792784452438354
******VALIDATION ACCURACY*************** :  0.47593748569488525
EPOCH :  4
TRAIN_ACCURACY :  1.1894357204437256
******VALIDATION ACCURACY*************** :  0.48625001311302185
EPOCH :  5
TRAIN_ACCURACY :  0.9924259185791016
******VALIDATION ACCURACY*************** :  0.49687498807907104
EPOCH :  6
TRAIN_ACCURACY :  0.8512016534805298
******VALIDATION ACCURACY*************** :  0.5018749833106995
EPOCH :  7
TRAIN_ACCURACY :  0.7450770139694214
******VALIDATION ACCURACY*************** :  0.49906250834465027
EPOCH :  8
TRAIN_ACCURACY :  0.6624607443809509
******VALIDATION ACCURACY*************** :  0.5006250143051147
EPOCH :  9
TRAIN_ACCURACY :  0.5963277816772461
******VALIDATION ACCURACY*************** :  0.49906250834465027